# BLACKJACK

A game of blackjack requires 52 to 416 cards. For simplicity, 52 cards are all we are gonna deal in or consider. The suits are namely: Hearts, Diamonds, Spades and Clubs. The goal of the game is to reach 21, and not beyond. The player can either hit or stand. So, basically it has other rules as well, which can be found [here](https://en.wikipedia.org/wiki/Blackjack)

In [1]:
# IMPORTS
import random

# GLOBAL VARIABLES SPECIFIC TO BLACKJACK
# card values per blackjack
card_values = {'Ace':11, 'Two':2, 'Three':3, 'Four':4, 'Five':5,
               'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10,
               'Jack':10, 'Queen':10, 'King':10}
# suits of cards
card_suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
# ranks of cards
card_ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')

In [2]:
class Card():
    def __init__(self, suit, rank):
        # the suit which card belongs to
        self.suit = suit
        # the rank of the card
        self.rank = rank
        
    def __str__(self):
        return f'{self.rank} of {self.suit}'

In [3]:
class Deck():
    def __init__(self, suits, ranks):
        # our deck as a list
        self.deck = []
        
        # Iterate through possible suit-rank combinations
        # add card to deck
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))
    
    def __str__(self):
        return f'Cards left: {len(self.deck)}, Cards: {[str(card) for card in self.deck]}'
    
    def shuffle(self):
        # shuffle deck using random
        random.shuffle(self.deck)
        
    def deal(self):
        # get a random index
        card_index = random.randrange(0, len(self.deck))
        # pop card from deck so that dealt card gets removed from the deck
        card = self.deck.pop(card_index)
        # return popped card
        return card

In [14]:
class Hand():
    def __init__(self, player_name):
        # set name of the player
        self.player_name = player_name
        # the cards in the hand, initialized to empty
        self.cards = []
        # the value of cards, initialized to zero
        self.value = 0
        # the number of aces in the hand
        self.aces = 0
        
    def add(self, card):
        # add card to hand
        self.cards.append(card)
        # increase card value
        self.value += card_values[card.rank]
        # see if it is ace, and increment aces counter when positive
        if card.rank == 'Ace':
            self.aces += 1
        
    def adjust_aces(self):
        # adjust to find best value if ace is present
        while self.value > 21 and self.aces > 0:
            self.value -= 10
            self.aces -= 1
            
    def show(self, all_cards=True):
        if all_cards:
            # show all cards
            print(f"\n{self.player_name}'s Hand:", *self.cards, sep='\n ')
        else:
            print(f"\n{self.player_name}'s Hand:", self.cards[0], sep='\n ')
            print(''.join('::Hidden Card::'))

In [5]:
class Chips():
    def __init__(self, total_chips=100):
        # total number of chips that a player has for bet, default 100
        self.total_chips = total_chips
        # current bet
        self.bet = 0
        
    def __str__(self):
        return f'{self.total_chips} available for bet'
        
    def compute(self, won):
        if won:
            self.total_chips += self.bet
        else:
            self.total_chips -= self.bet

In [6]:
def bet(chips):
    while True:
        try:
            bet = int(input('How much chips would you like to bet? '))
        except:
            print('It seems like you have entered an invalid input. Please try again.')
            continue
        else:
            if chips.total_chips < bet:
                print('Value of bet can not exceed the total chips you have. Please input again!')
                continue
            else:
                chips.bet = bet
                break

In [7]:
def hit(deck, hand):
    # deal a card and add it to hand
    hand.add(deck.deal())
    # adjust hand for ace to optimize score for best value
    hand.adjust_aces()

In [8]:
def hit_or_stand(deck, hand):
    while True:
        inp = input("Hit or Stand? Enter 'h' or 's'")
        
        if inp[0].lower() == 'h':
            hit(deck, hand)
            return 'hit'
        elif inp[0].lower() == 's':
            print("Player stands; Dealer is playing.")
            return 'stand'
        else:
            print("Sorry, please try again.")
            continue

In [9]:
def show_some(player, dealer):
    print("\nDealer's Hand:")
    print(" <card hidden>")
    print('',dealer.cards[1])  
    print("\nPlayer's Hand:", *player.cards, sep='\n ')
    
def show_all(player, dealer):
    print("\nDealer's Hand:", *dealer.cards, sep='\n ')
    print("Dealer's Hand =",dealer.value)
    print("\nPlayer's Hand:", *player.cards, sep='\n ')
    print("Player's Hand =",player.value)

In [10]:
def process_result(player, dealer, chips, who_won):
    if who_won.lower() == 'player':
        print('Player won!')
        chips.compute(True)
    elif who_won.lower() == 'dealer':
        print('Dealer won!')
        chips.compute(False)
    else:
        print('Dealer and Player tie!')

In [15]:
while True:
    # Print an opening statement
    print('Welcome to BlackJack! Get as close to 21 as you can without going over!\n\
    Dealer hits until she reaches 17. Aces count as 1 or 11.')
    
    # Create & shuffle the deck, deal two cards to each player
    deck = Deck(card_suits, card_ranks)
    deck.shuffle()
    
    player_hand = Hand('player')
    player_hand.add(deck.deal())
    player_hand.add(deck.deal())
    
    dealer_hand = Hand('dealer')
    dealer_hand.add(deck.deal())
    dealer_hand.add(deck.deal())
            
    # Set up the Player's chips
    player_chips = Chips()  # remember the default value is 100    
    
    # Prompt the Player for their bet
    bet(player_chips)
    
    # Show cards (but keep one dealer card hidden)
    player_hand.show()
    dealer_hand.show(False)
    # show_some(player_hand,dealer_hand)
    
    while True:  # recall this variable from our hit_or_stand function
        
        # Prompt for Player to Hit or Stand
        response = hit_or_stand(deck,player_hand) 
        
        # Show cards (but keep one dealer card hidden)
        player_hand.show()
        dealer_hand.show(False) 
        
        # If player's hand exceeds 21, run player_busts() and break out of loop
        if player_hand.value > 21:
            process_result(player_hand, dealer_hand, player_chips, 'dealer')
            break
            
        if response == 'stand':
            break

    # If Player hasn't busted, play Dealer's hand until Dealer reaches 17 
    if player_hand.value <= 21:
        
        while dealer_hand.value < 17:
            hit(deck, dealer_hand)    
    
        # Show all cards
        player_hand.show()
        dealer_hand.show()
        
        # Run different winning scenarios
        who_won = 'player' if dealer_hand.value > 21 or dealer_hand.value < player_hand.value else 'dealer' if dealer_hand.value > player_hand.value else 'None'
        
        process_result(player_hand, dealer_hand, player_chips, who_won)
    
    # Inform Player of their chips total 
    print("\nPlayer's winnings stand at",player_chips.total_chips)
    
    # Ask to play again
    new_game = input("Would you like to play another hand? Enter 'y' or 'n' ")
    
    if new_game[0].lower() == 'y':
        continue
    else:
        print("Thanks for playing!")
        break

Welcome to BlackJack! Get as close to 21 as you can without going over!
    Dealer hits until she reaches 17. Aces count as 1 or 11.
How much chips would you like to bet? 56

player's Hand:
 Three of Spades
 Four of Clubs

dealer's Hand:
 Five of Hearts
::Hidden Card::
Hit or Stand? Enter 'h' or 's'g
Sorry, please try again.
Hit or Stand? Enter 'h' or 's's
Player stands; Dealer is playing.

player's Hand:
 Three of Spades
 Four of Clubs

dealer's Hand:
 Five of Hearts
::Hidden Card::

player's Hand:
 Three of Spades
 Four of Clubs

dealer's Hand:
 Five of Hearts
 Nine of Hearts
 Five of Spades
Dealer won!

Player's winnings stand at 44
Would you like to play another hand? Enter 'y' or 'n' n
Thanks for playing!
